<a href="https://colab.research.google.com/github/DearZiZiZi/sql-agent-langchain/blob/main/UnitapAI_Anuar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Installing necessary libraries

In [1]:
!pip install langchain langchain-community psycopg2-binary openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 79.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 415.4/415.4 kB 29.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.40
    Uninstalling langchain-core-0.3.40:
      Successfully uninstalled langchain-core-0.3.40
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.19
    Uninstalling langchain-0.3.19:
      Successfully uninstalled langchain-0.3.19


# 2. Connecting with DataBase and customizing prompt

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.memory import ConversationBufferMemory
from langchain.tools import Tool
from langchain.sql_database import SQLDatabase
from langchain.agents import create_sql_agent
import os

# Set OpenAI API Key
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

# Define database connection with view support
db_url = "DataBase_name"
db = SQLDatabase.from_uri(db_url, include_tables=["university_details_v"], view_support=True)  # <- Added view_support=True

# Define LLM model
llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)

# Define custom prompt
custom_prompt = """
You are an SQL expert working with a specific database. You are also a consultant agent.
You must **only** use the following SQL view:

- **public.university_details_v** - general information
** - requirements, etc.

### **Available Columns:**
- 'university_name': name of the university
- 'rating': ranking in the world
- 'rating_country': ranking within its country
- 'category_name': name of the major category
- 'country': country name
- 'city_en': city name
- 'can_apply': can apply or not
- 'key_facts': key facts
- 'rating_information': ranking detailed information
- 'top_rating': top rating categories
- 'scholarship_type': scholarship type
- 'study_language': study language
- 'state': state name (if applicable)
- 'description': detailed description of the university
- 'university_link': university website link
- 'important_date_link': important dates link
- 'discount_link': discounts link
- 'description_en': detailed description in Russian
- 'tuition': tuition cost

**Do not use any other tables.**
"""

custom_table_info = {
    "university_details_v": """
        university_name TEXT,
        rating INTEGER,
        rating_country INTEGER,
        category_name TEXT,
        country TEXT,
        city_en TEXT,
        can_apply BOOLEAN,
        key_facts TEXT,
        rating_information TEXT,
        top_rating TEXT,
        scholarship_type TEXT,
        study_language TEXT,
        state TEXT,
        description TEXT,
        university_link TEXT,
        important_date_link TEXT,
        discount_link TEXT,
        description_en TEXT,
        tuition NUMERIC
    """
}

<ipython-input-2-57861cebe6da>:20: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model="gpt-4-turbo", temperature=0)


# 3. Creating SQL agent

In [3]:
# Create SQL Agent
sql_agent = create_sql_agent(
    llm=llm,
    db=db,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,  # Helps generate SQL queries
    verbose=True,
    custom_table_info=custom_table_info,
    extra_prompt_messages=[custom_prompt]  # Ensure it is passed as a list
)

# 4. Testing

In [4]:
# Example Usage
query = "I want to study in mew york, please recommend me the best choice in business. Max budget is 65,000$. give descriptions about the university"
response = sql_agent.run(query)
print(response)


<ipython-input-4-62f4219632b6>:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = sql_agent.run(query)




> Entering new SQL Agent Executor chain...
Action: sql_db_list_tables
Action Input: university_details_vThe table "university_details_v" seems relevant for the query about universities. I need to check the schema of this table to understand what columns are available, especially focusing on columns related to location, budget, and descriptions.

Action: sql_db_schema
Action Input: university_details_v
CREATE TABLE university_details_v (
	university_name VARCHAR, 
	rating INTEGER, 
	rating_country INTEGER, 
	category_name VARCHAR, 
	country VARCHAR, 
	city_en VARCHAR, 
	can_apply BOOLEAN, 
	key_facts JSON, 
	rating_information JSON, 
	top_rating university_entity_top_rating_enum, 
	scholarship_type university_entity_scholarship_type_enum, 
	study_language university_entity_study_language_enum, 
	state VARCHAR, 
	description TEXT, 
	university_link TEXT, 
	important_date_link TEXT, 
	discount_link TEXT, 
	description_en VARCHAR, 
	tuition INTEGER
)

/*
3 rows from university_details_v 